# 套件安裝

In [1]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai Pillow requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.3/734.3 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

# 套件讀取

In [2]:
import os
import gradio as gr
from PIL import Image
import requests
from io import BytesIO
import json
import torch # 確保 PyTorch 已安裝並可用

# LangChain 相關
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import ConversationalRetrievalChain

# OpenAI/Groq API 客戶端
from openai import OpenAI
from google.colab import userdata

import json
import codecs # 為了處理可能的 BOM

# 設定API金鑰

In [3]:
try:
    api_key = userdata.get('Groq')
    os.environ["OPENAI_API_KEY"] = api_key
    print("Groq API 金鑰已載入。")
except Exception as e:
    print(f"無法載入 Groq API 金鑰，請檢查 Colab Secrets 是否已設定 'Groq':{e}")

model_reader = "gemma2-9b-it" # 用於第一階段的意圖轉譯，選擇較輕量級的模型
model_generator = "llama3-70b-8192" # 用於第二階段的上下文生成，選擇功能更強的模型
base_url = "https://api.groq.com/openai/v1"

# 初始化 OpenAI 客戶端（兼容 Groq API）
client = OpenAI(base_url=base_url, api_key=api_key)

print(f"將使用的 Reader 模型: {model_reader}")
print(f"將使用的 Generator 模型: {model_generator}")

Groq API 金鑰已載入。
將使用的 Reader 模型: gemma2-9b-it
將使用的 Generator 模型: llama3-70b-8192


# 資料庫載入 (FAISS)

In [4]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

# 初始化 embedding 模型
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")

# --- 載入 MyGO!!!!! 圖片元數據資料庫 ---
image_metadata_db = []
try:
    with codecs.open('MyGO!!!!!_database.txt', 'r', encoding='utf-8-sig') as f:
        json_string_content = f.read()

    if not json_string_content.strip(): # 用 strip() 排除只包含空白字符的情況
        print("警告：'MyGO!!!!!_database.txt' 檔案內容為空或只包含空白字符。")
        image_metadata_db = []
    else:
        parsed_data = json.loads(json_string_content)
        image_metadata_db = parsed_data
        print(f"MyGO!!!!! 圖片元數據資料庫載入成功，共載入 {len(image_metadata_db)} 條記錄。")

except FileNotFoundError:
    print("錯誤：找不到 'MyGO!!!!!_database.txt' 檔案。請確保檔案已上傳至 Colab 環境。")
    image_metadata_db = []
except json.JSONDecodeError as e:
    print(f"錯誤：'MyGO!!!!!_database.txt' 檔案格式無效，無法解析 JSON：{e}")
    print("請檢查文件內容是否符合 JSON 格式！特別是逗號、引號和括號。")
    print(f"解析失敗的詳細錯誤：{e.msg} at line {e.lineno}, column {e.colno}")
    print("以下是嘗試解析的 JSON 字串片段 (前200字元)：")
    print(json_string_content[:200]) # 確保這裡能印出內容
    image_metadata_db = []
except Exception as e:
    print(f"載入或處理 MyGO!!!!!_database.txt 時發生未知錯誤：{e}")
    image_metadata_db = []


# 將圖片的多角度情境描述向量化，並建立 FAISS 索引
if image_metadata_db: # 只有當資料庫有內容時才進行embedding和FAISS建立
    texts_for_embedding = [item["multi_contextual_description"] for item in image_metadata_db]
    metadatas = image_metadata_db # 將整個元數據作為 document 的 metadata

    # 建立 FAISS 資料庫
    db = FAISS.from_texts(texts_for_embedding, embedding_model, metadatas=metadatas)
    retriever = db.as_retriever(search_kwargs={"k": 1}) # 檢索最相關的 1 個結果
    print(f"資料庫中共有 {len(image_metadata_db)} 張圖片元數據已載入並向量化。")
else:
    print("由於資料庫載入失敗或為空，FAISS 資料庫未建立。")
    db = None
    retriever = None

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

MyGO!!!!! 圖片元數據資料庫載入成功，共載入 10 條記錄。
資料庫中共有 10 張圖片元數據已載入並向量化。


# Prompt設計(兩階段思考)

In [5]:
# 第一階段：使用者意圖轉譯與提示精煉的 Prompt
system_prompt_reader = """你是一位精通 MyGO!!!!! 文化的資深工讀生，請根據使用者的提問，將其轉化為更具體、符合該圈子語境的圖片檢索提示。
請思考以下幾個方面並輸出結構化的文字，作為下一階段向量檢索的更精準輸入：
- 關鍵主題的精確提煉
- 情感或語氣的捕捉
- 對應特定文化梗或專有名詞的轉換
- 對圖片屬性的精確描述

請直接輸出精煉後的查詢，不要包含額外的說明。
範例輸入: 祥子後來怎麼樣了?
範例輸出: 豐川祥子 現狀 落寞 表情 崩壞

範例輸入: 燈平常都在做什麼?
範例輸出: 高松燈 日常 生活 練習

除了相關人物本身的語句，日常對話也可以提煉，主要為讀取訊息者的想法，供給下一階段的模型更精確的查詢。

範例輸入: 起司越多，洞越多；洞越多，起司越少；所以起司越多，起司就越少。
範例輸出: 邏輯崩壞　無理　奇怪的人
"""

# 第二階段：向量檢索與上下文生成 (主生成語言模型) 的 Prompt
system_prompt_generator = """你是動畫作品MyGO!!!!!相關衍生作品以及meme的狂熱粉絲，回答問題時經常使用一些人物說過的的台詞進行回應，有時甚至只使用台詞回應問題而且不會過多贅述，可以根據台詞簡短回答使用者的問題，請用台灣習慣的中文回應。

請根據使用者原始問題，從資料庫檢索到的最相關圖片描述，以MyGO!!!!!資深粉絲的角度，選擇最適合的圖片回覆使用者。你的選擇應該具備高度人性化與情境化，尤其是針對「MyGO!!!!!」這類特定文化圈，將更具備該圈子的語感和幽默感。
"""

# 修正後的 prompt_template_generator
prompt_template_generator = """你是一個MyGO!!!!!機器人。你擁有一個圖片資料庫，其中包含圖片名稱、URL、簡要描述和多重上下文描述。
當使用者提出問題時，請先檢查提供的「圖片資訊」。如果「圖片資訊」中有與問題高度相關的圖片，請優先**直接以 Markdown 圖片格式** 回答，例如：`![圖片描述](圖片URL)`。
其中，`圖片描述` 請使用該圖片的 `simple_description`。
如果有多張相關圖片，可以列出多張。
如果沒有找到高度相關的圖片資訊，則根據「圖片資訊」和你的知識，以中文文字回答問題。

圖片資訊：
{retrieved_image_description}

使用者問題：{original_question}

你的回答：
"""

# 核心RAG邏輯

In [6]:
def load_image_from_url(url):
    try:
        response = requests.get(url, timeout=10) # 增加 timeout 防止無限等待
        response.raise_for_status() # 檢查 HTTP 請求是否成功
        img = Image.open(BytesIO(response.content))
        return img
    except requests.exceptions.RequestException as e:
        print(f"圖片下載失敗: {e} (URL: {url})")
        return None
    except Exception as e:
        print(f"圖片處理失敗: {e} (URL: {url})")
        return None

chat_history = [] # 用於儲存聊天歷史，供 Gradio Chatbot 使用

def mygo_rag_model(user_input):
    global chat_history

    if retriever is None:
        return "初始化錯誤：向量資料庫未成功載入。請檢查 'MyGO!!!!!_database.txt' 檔案。", None, "" # 確保這個回傳三個值

    # --- 第一階段：使用者意圖轉譯與提示精煉 ---
    print(f"原始使用者問題：{user_input}")

    messages_reader = [
        {"role": "system", "content": system_prompt_reader},
        {"role": "user", "content": user_input},
    ]

    refined_query_text = ""
    try:
        response_reader = client.chat.completions.create(
            model=model_reader,
            messages=messages_reader,
            max_tokens=100,
            temperature=0.7
        )
        refined_query_text = response_reader.choices[0].message.content.strip()
        print(f"意圖轉譯後查詢：{refined_query_text}")

    except Exception as e:
        print(f"第一階段意圖轉譯失敗：{e}，將直接使用原始問題進行檢索。")
        refined_query_text = user_input

    # --- 第二階段：向量檢索與上下文生成 ---

    docs = retriever.get_relevant_documents(refined_query_text)

    if not docs:
        response_text = "抱歉，我找不到與您查詢相關的 MyGO!!!!! 圖片或資料。請嘗試更換關鍵字或提供更多細節。"
        # 在沒有圖片時，圖片輸出為 None
        return response_text, None, refined_query_text

    best_match_doc = docs[0]
    best_match_image_data = best_match_doc.metadata

    image_url = best_match_image_data.get("image_url")
    simple_description = best_match_image_data.get("simple_description", "MyGO!!!!!相關圖片")
    contextual_description = best_match_image_data.get("multi_contextual_description", best_match_image_data.get("simple_description", "無詳細描述"))


    # 強制模型以圖片形式回覆，並在 Markdown 中包含簡單描述和 URL
    response_text_for_llm = f"已找到以下相關圖片資訊：\n圖片名稱: {best_match_image_data.get('image_name')}\n簡要描述: {simple_description}\n詳細情境描述: {contextual_description}\n圖片URL: {image_url}\n請根據這些資訊，直接以 Markdown 圖片格式回覆"

    messages_generator = [
        {"role": "system", "content": system_prompt_generator},
        {"role": "user", "content": response_text_for_llm},
    ]

    try:
        response_generator = client.chat.completions.create(
            model=model_generator,
            messages=messages_generator,
            max_tokens=250,
            temperature=0.8
        )
        llm_response_content = response_generator.choices[0].message.content.strip()

        response_text = f"![{simple_description}]({image_url})"

    except Exception as e:
        print(f"第二階段生成回覆失敗：{e}，將僅顯示圖片和預設文字。")
        response_text = f"![{simple_description}]({image_url})\n\n這張圖片來自MyGO!!!!!的世界，希望您喜歡！"

    img = load_image_from_url(image_url)

    # 確保回傳的順序是 (機器人回覆文字, 圖片物件, 轉譯查詢文字)
    return response_text, img, refined_query_text

# Gradio

In [8]:
def respond_to_gradio(message, history):
    # mygo_rag_model 回傳: (response_text, img, refined_query_text)
    response_text, img, refined_query = mygo_rag_model(message)

    # Gradio Chatbot 期望的 history 格式是 [ [user_msg, bot_msg], ... ]
    # 所以我們直接將新的對話回合以列表形式添加到 history 中
    history.append([message, response_text]) # 將使用者訊息和機器人回覆作為一個列表添加到歷史中

    # 回傳 chatbot 歷史、清空的輸入框、以及轉譯查詢文字
    return history, "", refined_query

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# MyGO!!!!! 樂團工讀生機器人")
    gr.Markdown("請輸入您想查詢的 MyGO!!!!! 相關問題，機器人會為您提供資訊和相關圖片。")

    # 第一階段轉譯關鍵詞 (只顯示，不可編輯)
    refined_query_output = gr.Textbox(label="第一階段轉譯檢索關鍵字", lines=1, interactive=False)

    # 聊天歷史窗口 (可顯示圖片，不可編輯)
    chatbot = gr.Chatbot(label="聊天歷史", height=500, show_copy_button=True)

    # 使用者輸入窗口 (可編輯，用於輸入問題)
    msg = gr.Textbox(placeholder="開始聊天吧...", label="使用者輸入區域")

    # 範例問題 (點擊後自動填充到使用者輸入窗口)
    gr.Examples(
        examples=[
            "Tomorin出擊!",
            "咕咕嘎嘎",
            "一輩子警察",
            "祥子加入了哪個新樂團？",
            "Crychic解散的原因是什麼？"
        ],
        inputs=msg,
        # 清空聊天歷史和轉譯關鍵詞 (可選，但通常點擊範例時會希望清空)
        outputs=[chatbot, msg, refined_query_output],
        fn=lambda x,y: (None, x, None), # 清空歷史，並將範例填入msg
        cache_examples=False,
    )

    # 清除按鈕 (可選)
    clear = gr.ClearButton([msg, chatbot, refined_query_output])

    # 事件綁定：當使用者輸入並提交時觸發 respond_to_gradio 函數
    # inputs: 提供給 respond_to_gradio 的參數
    # outputs: respond_to_gradio 的回傳值將更新到這些元件
    msg.submit(
        fn=respond_to_gradio,
        inputs=[msg, chatbot], # 傳入使用者輸入和當前的聊天歷史
        outputs=[chatbot, msg, refined_query_output] # 更新聊天歷史、清空使用者輸入、更新轉譯關鍵字
    )

# 啟動 Gradio 介面
if __name__ == "__main__":
    demo.launch(debug=True, share=True)

<ipython-input-8-626093429>:21: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="聊天歷史", height=500, show_copy_button=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://15ea048420b11b78a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


原始使用者問題：Tomorin好可愛
意圖轉譯後查詢：Tomorin 可愛  表情
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://15ea048420b11b78a2.gradio.live
